In [3]:
#word selector
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
import os

torch.cuda.empty_cache()

def get_words(image_dir):
    image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(('.jpg', '.png'))]
    for num, image_path in enumerate(image_paths):
        print(f"Progress: {num+1}/{len(image_paths)}", end="\r")
        image = Image.open(image_path).convert("RGB")
        input = processor(text=wordlist_og, images=image, return_tensors='pt', padding=True)
        input = {key: val.to(device) for key, val in input.items()}
        output = model(**input)
        logits_per_image = output.logits_per_image
        probs = logits_per_image.softmax(dim=1)
        sorted_probs, sorted_indices = torch.sort(probs, descending=True)
        i = 0
        this_prob = sorted_probs[0,i].item()
        while this_prob > threshold:
            index = sorted_indices[0,i].item()
            selected_indices.add(index)
            i += 1
            this_prob = sorted_probs[0,i].item()

threshold = 0.001
selected_indices = set()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

image_dir1 = 'Images/codenames_og'
image_dir2 = 'Images/coco_val2017/val2017'

with open("wordlist/wordlist_gh_allx.txt", "r", encoding="UTF-8") as f:
    wordlist_og = [line.strip() for line in f]

get_words(image_dir1)
#get_words(image_dir2)
            
new_wordlist = []
for index in selected_indices:
    new_wordlist.append(wordlist_og[index])

print(f"Old wordlist: {len(wordlist_og)}, New wordlist: {len(new_wordlist)}")

with open("wordlist/selected_words_point001.txt", "w") as file:
    for word in new_wordlist:
        file.write(word + "\n")

Old wordlist: 3180, New wordlist: 1615
